# Identification of enhancers case study 

This section will present a comparative analysis to demonstrate the application and performance of proPythia for addressing sequence-based prediction problems.

We'll try to replicate one of the [BioSeq-Analysis](https://academic.oup.com/nar/article/47/20/e127/5559689?login=true) case studies for identifying [enhancers](https://academic.oup.com/bioinformatics/article/32/3/362/1744331?login=true#btv604-M1).

In [1]:
%load_ext autoreload

import sys
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, matthews_corrcoef
from sklearn.preprocessing import StandardScaler

sys.path.append('../../../../src/')
from propythia.shallow_ml import ShallowML

from descriptors import DNADescriptor

2022-04-06 18:02:45.306222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-06 18:02:45.306238: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0


2022-04-06 18:02:49.019557: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-06 18:02:49.019578: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-06 18:02:49.019592: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-04-06 18:02:49.019766: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import csv

def write_dict_to_csv(d: dict, filename: str):
    """
    Writes a dictionary to a csv file.
    """
    with open(filename, 'w') as csv_file:
        writer = csv.writer(csv_file)
        headers = ["id", "sequence"]
        writer.writerow(headers)
        for key, val in d.items():
            writer.writerow([key, val])

In [3]:
from sequence import ReadDNA

dna = ReadDNA()
dna.read_fasta_in_folder('enhancer_dataset')
for i in dna.d:
    write_dict_to_csv(dna.d[i], f'enhancer_dataset/{i}.csv')

This dataset has **742** weak enhancers, **742** strong enhancers, and **1484** non-enhancers.

In [4]:
strong_file =  r'enhancer_dataset/strong.csv'
weak_file =  r'enhancer_dataset/weak.csv'
non_file =  r'enhancer_dataset/non-enhancers.csv'

strong = pd.read_csv(strong_file)
weak = pd.read_csv(weak_file)
non = pd.read_csv(non_file)

print('strong', strong.shape)
print('weak', weak.shape)
print('non', non.shape)

strong (742, 2)
weak (742, 2)
non (1484, 2)


To calculate features, and to be more easy, we create a function to calculate features, calculating all available DNA features.

In [5]:
def calculate_feature(data):
    list_feature = []
    count = 0
    for seq in data['sequence']:
        res = {'sequence': seq}
        dna = DNADescriptor(seq)
        feature = dna.get_all_descriptors()
        res.update(feature)
        list_feature.append(res)
        
        # print progress every 100 sequences
        if count % 100 == 0:
            print(count, '/', len(data))

        count += 1
    print("Done!")
    df = pd.DataFrame(list_feature)
    return df

strong_feature = calculate_feature(strong)
weak_feature = calculate_feature(weak)
non_feature = calculate_feature(non)

0 / 742
100 / 742
200 / 742
300 / 742
400 / 742
500 / 742
600 / 742
700 / 742
Done!
0 / 742
100 / 742
200 / 742
300 / 742
400 / 742
500 / 742
600 / 742
700 / 742
Done!
0 / 1484
100 / 1484
200 / 1484
300 / 1484
400 / 1484
500 / 1484
600 / 1484
700 / 1484
800 / 1484
900 / 1484
1000 / 1484
1100 / 1484
1200 / 1484
1300 / 1484
1400 / 1484
Done!


- In the dataframe, each row is a sequence and each column is a feature.
- There are 19 different features for each sequence.

In [6]:
# put labels for each dataset   
strong_feature['label'] = 2
weak_feature['label'] = 1
non_feature['label'] = 0

print(strong_feature.shape)
print(weak_feature.shape)
print(non_feature.shape)

(742, 21)
(742, 21)
(1484, 21)


In [7]:
dataset = pd.concat([strong_feature, weak_feature, non_feature])

fps_y = dataset['label']
fps_x = dataset.loc[:, dataset.columns != 'label']
fps_x = fps_x.loc[:, fps_x.columns != 'sequence']

print(fps_x.shape)

(2968, 19)


In [8]:
no_need_normalization = ["length", "at_content", "gc_content"]

need_dict_normalization = ["nucleic_acid_composition", "enhanced_nucleic_acid_composition","dinucleotide_composition","trinucleotide_composition","k_spaced_nucleic_acid_pairs","kmer","PseDNC", "PseKNC"]

need_list_normalization = ["nucleotide_chemical_property", "accumulated_nucleotide_frequency", "DAC", "DCC", "DACC", "TAC","TCC","TACC"]

def normalize_dict(d, field):
    df = pd.json_normalize(d)
    df.columns = [str(field) + "_" + str(i) for i in df.columns]
    
    for f in df.columns:
        if isinstance(df[f][0], dict):
            df = pd.concat([df, normalize_dict(df[f], f)], axis=1)
            df.drop(f, axis=1, inplace=True)
    return df

def normalize_list(l, field):
    df = pd.DataFrame(l.to_list())
    df.columns = [str(field) + "_" + str(i) for i in df.columns]
    
    for f in df.columns:
        if isinstance(df[f][0], list):
            df = pd.concat([df, normalize_list(df[f], f)], axis=1)
            df.drop(f, axis=1, inplace=True)
    return df

new_fps_x = pd.DataFrame()

for col in fps_x.columns:
    if col in need_dict_normalization:
        new_fps_x = pd.concat([new_fps_x, normalize_dict(fps_x[col], col)], axis=1)
    elif col in need_list_normalization:
        new_fps_x = pd.concat([new_fps_x, normalize_list(fps_x[col], col)], axis=1)
    else:
        new_fps_x[col] = fps_x[col].to_numpy()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(new_fps_x, fps_y, stratify=fps_y)

# standard scaler article does not refer scaling and do not validate in x_test, however, we do it anyway

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# open a ShallowML object
ml = ShallowML(X_train, X_test, y_train, y_test, report_name=None, columns_names=new_fps_x.columns)

# define param grid as article, here we will search in 100, 200 and 500 estimators
param_grid = [{'clf__n_estimators': [100, 200, 500], 'clf__max_features': ['sqrt']}]

# train_best_model will perform a GRIDSEARCHCV optimizing MCC with a cv = 10
# best_rf_model_enhancers = ml.train_best_model(model_name=None, model='rf', score=make_scorer(matthews_corrcoef), param_grid=param_grid, cv=10)

best_rf_model_enhancers = ml.train_best_model(model_name=None,model='svm', scaler=None,
                 score=make_scorer(matthews_corrcoef),
                 cv=10, optType='gridSearch', param_grid=None,
                 n_jobs=10,
                 random_state=1, n_iter=15, refit=True)

performing gridSearch...


KeyboardInterrupt: 

In [ ]:
scores, report, cm, cm2 = ml.score_testset(best_rf_model_enhancers)
print(report)
print(cm)  
scores

              precision    recall  f1-score   support

           0       0.69      0.87      0.77       371
           1       0.39      0.05      0.09       185
           2       0.52      0.70      0.59       186

    accuracy                           0.62       742
   macro avg       0.53      0.54      0.48       742
weighted avg       0.57      0.62      0.56       742

[[324   6  41]
 [ 96   9  80]
 [ 48   8 130]]


{'Accuracy': 0.623989218328841,
 'MCC': 0.3917814923704463,
 'log_loss': 0.8292365194048797,
 'f1 score weighted': 0.5568926567744779,
 'f1 score macro': 0.4846173899895776,
 'f1 score micro': 0.623989218328841,
 'roc_auc ovr': 0.7777556648074809,
 'roc_auc ovo': 0.754091381354792,
 'precision': 0.5735473309279806,
 'recall': 0.623989218328841}